In [1]:
import os
import numpy as np

from tensorflow.keras.models import load_model
from tensorflow.keras.datasets.mnist import load_data

from surprise_adequacy import SurpriseAdequacyConfig, DSA, LSA

In [2]:
CLIP_MIN = -0.5
CLIP_MAX = 0.5

In [3]:
(x_train, _), (x_test, _) = load_data(path=os.path.expanduser('~/dev/data/mnist/mnist.npz'))

In [4]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train = (x_train / 255.0) - (1.0 - CLIP_MAX)
x_test = (x_test / 255.0) - (1.0 - CLIP_MAX)

In [5]:
# model = load_model('../../sadl/model/model_mnist.h5')
model = load_model('./models/model_01/model_mnist.h5')

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 activation (Activation)     (None, 26, 26, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 activation_1 (Activation)   (None, 24, 24, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                        

In [14]:
config = SurpriseAdequacyConfig(saved_path='./tmp', is_classification=True, num_classes=10, 
                                layer_names=["activation_3"], ds_name="mnist")

In [15]:
# calculating LSA

sa = LSA(model=model, train_data=x_train, config=config)
sa.prep(use_cache=False)

surprises, predictions = sa.calc(target_data=x_test, use_cache=False, ds_type='test')

surprises = np.argsort(surprises)[::-1]

np.save('./tmp/lsa.npy', surprises)

Calculating the ats for train dataset
469/469 [==============================] - 22s 48ms/step
Layer: activation_3
[train] Saved the ats and predictions to ./tmp/mnist_train_activation_3_ats.npy and ./tmp/mnist_train_pred.npy


kde: 100%|██████████| 10/10 [00:00<00:00, 249.06it/s]

Ignoring the activations of 0 traces as their variance is not high enough.
Calculating the ats for test dataset


79/79 [==============================] - 5s 59ms/step
Layer: activation_3
[test] Saved the ats and predictions to ./tmp/mnist_test_activation_3_ats.npy and ./tmp/mnist_test_pred.npy
[test] Calculating LSA


In [16]:
# calculating DSA

sa = DSA(model=model, train_data=x_train, config=config)
sa.prep(use_cache=False)

surprises, predictions = sa.calc(target_data=x_test, use_cache=False, ds_type='test')

surprises = np.argsort(surprises)[::-1]

np.save('./tmp/dsa.npy', surprises)

Calculating the ats for train dataset
469/469 [==============================] - 23s 49ms/step
Layer: activation_3
[train] Saved the ats and predictions to ./tmp/mnist_train_activation_3_ats.npy and ./tmp/mnist_train_pred.npy
Calculating the ats for test dataset
79/79 [==============================] - 4s 46ms/step
Layer: activation_3
[test] Saved the ats and predictions to ./tmp/mnist_test_activation_3_ats.npy and ./tmp/mnist_test_pred.npy
[test] Calculating DSA
[<class 'surprise_adequacy.DSA'>] Using 60000 train samples
